In [24]:
from ortools.sat.python import cp_model

In [25]:
MAX_VALUE = 99999999

N = 0
M = 0
classes = {}
room_capacity = {}

def read_input(file_name):
    global N, M, classes, room_capacity

    with open(file_name) as file:
        lines = [line.rstrip() for line in file]

    N, M = tuple(int(x) for x in lines[0].split())

    for i in range(1, N+1):
        num_periods, teacher, num_pupils = tuple(int(x) for x in lines[i].split())
        classes[i] = {
            'num_periods': num_periods,
            'teacher': teacher,
            'num_pupils': num_pupils,
        }

    room_capacity = {idx+1: int(x) for idx, x in enumerate(lines[-1].split())}

    if len(classes) != N or len(room_capacity) != M:
        raise Exception('Error when reading data!')

read_input('./data/input0.txt')

In [26]:
classes

{1: {'num_periods': 4, 'teacher': 1, 'num_pupils': 15},
 2: {'num_periods': 4, 'teacher': 1, 'num_pupils': 18},
 3: {'num_periods': 4, 'teacher': 1, 'num_pupils': 15},
 4: {'num_periods': 2, 'teacher': 2, 'num_pupils': 18},
 5: {'num_periods': 4, 'teacher': 2, 'num_pupils': 11},
 6: {'num_periods': 3, 'teacher': 1, 'num_pupils': 15},
 7: {'num_periods': 2, 'teacher': 2, 'num_pupils': 27},
 8: {'num_periods': 3, 'teacher': 2, 'num_pupils': 18},
 9: {'num_periods': 4, 'teacher': 1, 'num_pupils': 13},
 10: {'num_periods': 3, 'teacher': 1, 'num_pupils': 10}}

In [27]:
room_capacity

{1: 20, 2: 20}

# MODEL

In [28]:
model = cp_model.CpModel()

In [29]:
# VARIABLES & DOMAINS
class_is_chosen = {} # Lớp có được xếp hay không, 1 nếu được xếp, 0 nếu bỏ
class_session = {} # Buổi học, D = {1, ..., 10}
class_starting = {} # Tiết học bắt đầu (trong buổi), D = {1, ..., 6}
class_room = {} # Phòng học, D = {1, ..., M}

for i in range(1, N + 1):
    class_is_chosen[i] = model.NewBoolVar(f'class_is_chosen[{i}]')
    class_session[i] = model.NewIntVar(1, 10, f'class_session[{i}]')
    class_starting[i] = model.NewIntVar(1, 6, f'class_starting[{i}]')
    class_room[i] = model.NewIntVar(1, M, f'clas_room[{i}]')

In [31]:
# CONSTRAINTS

for i in range(1, N + 1):

    # Sĩ số lớp được chọn ít hơn kích thước phòng
    temp_room_capacity = model.NewIntVar(0, MAX_VALUE, f'temp_room_capacity[{i}]')
    model.AddElement(class_room[i], room_capacity, temp_room_capacity)
    model.Add(
        temp_room_capacity >= classes[i]['num_pupils']
    ).OnlyEnforceIf(class_is_chosen[i])

    # Lớp học giới hạn trong một buổi
    model.Add(class_starting[i] + classes[i]['num_periods'] - 1 <= 6)

for i in range(1, N + 1):
    for j in range(1, N + 1):
        if i == j:
            continue

        i_same_session_j = model.NewBoolVar(f'i_same_session_j[{i},{j}]')
        model.Add(class_session[i] == class_session[j]).OnlyEnforceIf(i_same_session_j)
        i_start_before_j = model.NewBoolVar(f'i_start_before_j[{i},{j}]')
        model.Add(class_starting[i] <= class_starting[j]).OnlyEnforceIf(i_start_before_j)
        i_end_after_j_start = model.NewBoolVar(f'i_end_after_j_start[{i},{j}]')
        model.Add(class_starting[i] + classes[i]['num_periods'] - 1 >= class_starting[j]).OnlyEnforceIf(i_end_after_j_start)
        i_overlapped_by_j = sum([i_same_session_j, i_start_before_j, i_end_after_j_start])

        # Hai lớp cùng giáo viên không chồng lên nhau
        if classes[i]['teacher'] == classes[j]['teacher']:
            model.Add(
                i_overlapped_by_j != 3
            ).OnlyEnforceIf(
                class_is_chosen[i] 
            ).OnlyEnforceIf(
                class_is_chosen[j]
            )

        # Hai lớp xếp chồng giờ không cùng phòng
        i_same_room_j = model.NewBoolVar(f'i_same_room_j[{i},{j}]')
        model.Add(class_room[i] == class_room[j]).OnlyEnforceIf(i_same_room_j)

        model.Add(
            i_overlapped_by_j != 3
        ).OnlyEnforceIf(
            class_is_chosen[i] 
        ).OnlyEnforceIf(
            class_is_chosen[j]
        ).OnlyEnforceIf(
            i_same_room_j
        )

In [32]:
# OBJECTIVES
model.Maximize(
    sum([class_is_chosen[i] for i in class_is_chosen])
)

In [33]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [34]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for i in class_is_chosen:
        print(solver.Value(class_is_chosen[i]))
else:
    print('No solution found.')

0
0
0
0
0
0
0
0
0
0
